In [4]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

NUM_EPOCHS = 100
INIT_LR = 5e-3
COLORS = 1
KERN = 3

#create an object of the sequential class
model = Sequential()

model.add(Conv2D(32, (KERN, KERN), input_shape = (64, 64, COLORS)))
#add the pooling layer
model.add(MaxPooling2D(pool_size = (2, 2), strides= (2,2)))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv2D(32, (KERN, KERN)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Conv2D(64, (KERN, KERN), activation='relu'))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

#convert the pooled images into a 1D features vector
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
#initilize output layer
model.add(Dense(6, activation = 'softmax')) #6 labels

opt = SGD(lr=INIT_LR, momentum=0.9)
#opt = 'adam'
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])


Using TensorFlow backend.


In [5]:
batch_size = 16
train_datagen = ImageDataGenerator( #rescale = 1./255 
featurewise_center=False, # set input mean to 0 over the dataset
samplewise_center=False, # set each sample mean to 0
featurewise_std_normalization=False, # divide inputs by std of the dataset
samplewise_std_normalization=False, # divide each input by its std
#zca_whitening=False, # apply ZCA whitening
rotation_range=1, # randomly rotate images in the range (degrees, 0 to 180)
width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
height_shift_range=0.1, # randomly shift images vertically (fraction of total height)
horizontal_flip=False, # randomly flip images
vertical_flip=False) # randomly flip images


test_datagen = ImageDataGenerator( #rescale = 1./255 
#featurewise_center=False, # set input mean to 0 over the dataset
samplewise_center=False, # set each sample mean to 0
#featurewise_std_normalization=False, # divide inputs by std of the dataset
samplewise_std_normalization=False, # divide each input by its std
#zca_whitening=False, # apply ZCA whitening
#rotation_range=1, # randomly rotate images in the range (degrees, 0 to 180)
#width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
#height_shift_range=0.1, # randomly shift images vertically (fraction of total height)
#horizontal_flip=False, # randomly flip images
vertical_flip=False) # randomly flip images
#rescale = 1./255)

td = 'Marcel-train'
TRAIN_IMG_CNT = 4872
#td = 'more-train'
#TRAIN_IMG_CNT = 19488
VALIDATION_CNT = 659

training_set = train_datagen.flow_from_directory(td, 
                                                 target_size = (64, 64), 
                                                 batch_size = batch_size,
                                                 color_mode = 'grayscale',
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Marcel-Test', 
                                            target_size = (64, 64), 
                                            batch_size = batch_size, 
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 4872 images belonging to 6 classes.
Found 659 images belonging to 6 classes.


In [6]:
def poly_decay(epoch):
	# initialize the maximum number of epochs, base learning rate,
	# and power of the polynomial
	maxEpochs = NUM_EPOCHS
	baseLR = INIT_LR
	power = 1.0
 
	# compute the new learning rate based on polynomial decay
	alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
 
	# return the new learning rate
	return alpha

In [7]:
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

early_stop_monitor = EarlyStopping(patience = 5)
callbacks = [
    LearningRateScheduler(poly_decay),
#            early_stop_monitor
    ]


In [5]:

model.fit_generator(training_set, steps_per_epoch = TRAIN_IMG_CNT / batch_size, epochs = NUM_EPOCHS,
                    callbacks=callbacks,
                    validation_data = test_set, validation_steps = VALIDATION_CNT / batch_size)
model.save_weights('1st_run.h5') 

Epoch 1/100
305/304 [==============================] - 10s 33ms/step - loss: 1.5534 - acc: 0.3846 - val_loss: 1.9125 - val_acc: 0.2261
Epoch 2/100
305/304 [==============================] - 9s 29ms/step - loss: 1.3392 - acc: 0.4508 - val_loss: 1.9543 - val_acc: 0.1836
Epoch 3/100
305/304 [==============================] - 9s 28ms/step - loss: 1.2720 - acc: 0.4738 - val_loss: 2.1465 - val_acc: 0.1608
Epoch 4/100
305/304 [==============================] - 8s 28ms/step - loss: 1.2366 - acc: 0.4799 - val_loss: 2.3919 - val_acc: 0.2322
Epoch 5/100
305/304 [==============================] - 9s 28ms/step - loss: 1.2248 - acc: 0.4852 - val_loss: 2.5326 - val_acc: 0.2640
Epoch 6/100
305/304 [==============================] - 9s 29ms/step - loss: 1.1046 - acc: 0.5305 - val_loss: 1.9249 - val_acc: 0.3111
Epoch 7/100
305/304 [==============================] - 9s 29ms/step - loss: 1.0311 - acc: 0.5791 - val_loss: 2.2433 - val_acc: 0.3581
Epoch 8/100
305/304 [==============================] - 9s 31m

In [8]:
import keras
print(keras.__version__)

2.3.1
